In [207]:
from scipy.optimize import *
import pandas as pd 
import numpy as np 
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from pulp import *
import plotly.express as px 
import plotly.io as pio 
pio.renderers.default = "browser"

In [200]:
def meanRetAn(data):             
    Result = 1
    
    for i in data:
        Result *= (1+i)
        
    Result = Result**(1/float(len(data)/52))-1
     
    return(Result)

In [193]:
df = pd.read_csv("WeeklyReturns.csv", index_col="Date")

In [210]:
from tqdm import tqdm
from itertools import combinations
samples = []
tickers = df.columns[1:20]
for i in tqdm(range(2,len(tickers)+1,2)):
    samples.extend([list(x) for x in combinations(tickers, i)])

100%|██████████| 9/9 [00:00<00:00, 26.03it/s]


In [212]:
len(samples)

262143

In [211]:
from sklearn.decomposition import PCA
pca = PCA()
PDI_dict = {}
for i,y in zip(samples,range(len(samples))):
    n_assets = len(i)
    portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
    port_weekly_return = df[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)
    ann_ret = meanRetAn(list(port_weekly_return))
    an_cov = df[i].cov()
    port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(52)
    corr_matrix = np.array(df[i].cov())
    principalComponents = pca.fit(corr_matrix)
    PDI = 2*sum(x*y for x,y in zip(principalComponents.explained_variance_ratio_,range(1,len(principalComponents.explained_variance_ratio_)+1,1)))-1
    PDI_dict[y] = {}
    PDI_dict[y]["PDI_INDEX"] = PDI
    PDI_dict[y]["Number of Assets"] = len(i)
    PDI_dict[y]["Assets"] = i
    PDI_dict[y]["Sharpe Ratio"] = ann_ret/port_std
    PDI_dict[y]["Annual Return"] = ann_ret
    PDI_dict[y]["Annual STD"] = port_std

KeyboardInterrupt: 

In [205]:
PDI_DF = pd.DataFrame(PDI_dict).T
PDI_DF["Number of Assets"] = PDI_DF["Number of Assets"].astype(int)
PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)

In [208]:
fig = px.scatter(PDI_DF,  x="PDI_INDEX", y="Sharpe Ratio", color="Number of Assets")
fig.show()


In [11]:
tt = [0]
tt.extend([x for x in range(2,11)])

In [12]:
tt

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10]